<a href="https://colab.research.google.com/github/anyuanay/medium/blob/main/src/working_huggingface/Working_with_HuggingFace_ch2_Fine_Tuning_NER_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial: Working with Hugging Face Models and Datasets
## Chapter 2: Named Entity Recognition (NER) using Models in Hugging Face
### Lesson 2.3: Fine-tuning the pre-trained bert-base-NER model for NER

In this lesson, we will use the prepared WNUT17 dataset for fine-tuning the pre-trained bert-base-NER model for the named entity recoginition (NER) task.

# Install Transformers and Datasets from Hugging Face

In [1]:
# Transformers installation
! pip install -q transformers[torch] datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.2 MB/s eta 0:00:00


## Load the WNUT 2017 dataset


In [2]:
from datasets import load_dataset

wnut = load_dataset('wnut_17')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

# List the Tag Names in the WNUT 2017 Dataset

In [3]:
tag_names = wnut["test"].features["ner_tags"].feature.names
tag_names

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

# Load the Toknenizer of the bert-base-NER Model

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

# Tokenize the WNUT 17 Dataset

In [5]:
def tokenize_and_align_tags(records):
    # Tokenize the input words. This will break words into subtokens if necessary.
    # For instance, "ChatGPT" might become ["Chat", "##G", "##PT"].
    tokenized_results = tokenizer(records["tokens"], truncation=True, is_split_into_words=True)

    input_tags_list = []

    # Iterate through each set of tags in the records.
    for i, given_tags in enumerate(records["ner_tags"]):
        # Get the word IDs corresponding to each token. This tells us to which original word each token corresponds.
        word_ids = tokenized_results.word_ids(batch_index=i)

        previous_word_id = None
        input_tags = []

        # For each token, determine which tag it should get.
        for wid in word_ids:
            # If the token does not correspond to any word (e.g., it's a special token), set its tag to -100.
            if wid is None:
                input_tags.append(-100)
            # If the token corresponds to a new word, use the tag for that word.
            elif wid != previous_word_id:
                input_tags.append(given_tags[wid])
            # If the token is a subtoken (i.e., part of a word we've already tagged), set its tag to -100.
            else:
                input_tags.append(-100)
            previous_word_id = wid

        input_tags_list.append(input_tags)

    # Add the assigned tags to the tokenized results.
    # Hagging Face trasformers use 'labels' parameter in a dataset to compute losses.
    tokenized_results["labels"] = input_tags_list

    return tokenized_results


In [6]:
tokenized_wnut = wnut.map(tokenize_and_align_tags, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [7]:
for key in wnut['train'][1]:
    print(key, ":", wnut['train'][0][key])

id : 0
tokens : ['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.']
ner_tags : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
for key in tokenized_wnut['train'][1]:
    print(key, ":", tokenized_wnut['train'][1][key])

id : 1
tokens : ['From', 'Green', 'Newsfeed', ':', 'AHFA', 'extends', 'deadline', 'for', 'Sage', 'Award', 'to', 'Nov', '.', '5', 'http://tinyurl.com/24agj38']
ner_tags : [0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
input_ids : [101, 1622, 2565, 3128, 8124, 1174, 131, 138, 13561, 1592, 8559, 17638, 1111, 15204, 1698, 1106, 14152, 119, 126, 8413, 131, 120, 120, 4296, 2149, 1233, 119, 3254, 120, 1572, 8517, 3361, 23249, 102]
token_type_ids : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
attention_mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
labels : [-100, 0, 0, 0, -100, -100, 0, 5, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100]


# Create Data Collator

Now create a batch of examples using [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorWithPadding). It's more efficient to *dynamically pad* the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [9]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Fine Tune the bert-base-NER Model
The above steps prepared the WNUT datasets. Now, let us fine tine the bert-base-NER model.

## Create Maps from Tag Ids to Tag Names

Before we start fine-tuning the model, we create a map of the expected tag ids to their tag names with `id2label` and `label2id`:

In [11]:
id2label = dict(enumerate(tag_names))
id2label

{0: 'O',
 1: 'B-corporation',
 2: 'I-corporation',
 3: 'B-creative-work',
 4: 'I-creative-work',
 5: 'B-group',
 6: 'I-group',
 7: 'B-location',
 8: 'I-location',
 9: 'B-person',
 10: 'I-person',
 11: 'B-product',
 12: 'I-product'}

In [13]:
label2id = dict(zip(id2label.values(), id2label.keys()))
label2id

{'O': 0,
 'B-corporation': 1,
 'I-corporation': 2,
 'B-creative-work': 3,
 'I-creative-work': 4,
 'B-group': 5,
 'I-group': 6,
 'B-location': 7,
 'I-location': 8,
 'B-person': 9,
 'I-person': 10,
 'B-product': 11,
 'I-product': 12}

## Load the Pre-Trained Model by Passing the Maps between Ids and Names
Load the bert-base-NER model with AutoModelForTokenClassification along with the number of expected tags, and the tag mappings:

In [56]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "dslim/bert-base-NER", num_labels=len(id2label), id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([13]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768])

## Train the Model

We will use Hugging Face [Trainer](https://huggingface.co/docs/transformers/main/en/tasks/../training#train-with-pytorch-trainer). The Trainer API supports a wide range of training options and features such as logging, gradient accumulation, and mixed precision.

First, create a TrainingArguments instance. We will begin with the default training hyperparameters and experiment with different settings later. The only required parameter is `output_dir`. We can specify a local one.

In [16]:
training_args = TrainingArguments(
    output_dir="my_finetuned_wnut_model",
)

Let's go!

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
500,0.023300
1000,0.012800


TrainOutput(global_step=1275, training_loss=0.015794914469999425, metrics={'train_runtime': 165.7287, 'train_samples_per_second': 61.438, 'train_steps_per_second': 7.693, 'total_flos': 289317962396700.0, 'train_loss': 0.015794914469999425, 'epoch': 3.0})

## Inference on an Example

Prepare an example text for inference.

In [20]:
text = wnut['test'][1]['tokens']
text

['&',
 'gt',
 ';',
 '*',
 'Police',
 'last',
 'week',
 'evacuated',
 '80',
 'villagers',
 'from',
 'Waltengoo',
 'Nar',
 'where',
 'dozens',
 'were',
 'killed',
 'after',
 'a',
 'series',
 'of',
 'avalanches',
 'hit',
 'the',
 'area',
 'in',
 '2005',
 'in',
 'the',
 'south',
 'of',
 'the',
 'territory',
 '.']

The simplest way to try out the finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Let us load the model and tokenizer from the saved location.

In [27]:
from transformers import pipeline

model = AutoModelForTokenClassification.from_pretrained("my_finetuned_wnut_model/checkpoint-1000")
tokenizer = AutoTokenizer.from_pretrained("my_finetuned_wnut_model/checkpoint-1000")

classifier = pipeline("ner", model=model, tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Classify the text.

In [29]:
classifier(" ".join(text))

[{'entity': 'B-location',
  'score': 0.9942986,
  'index': 13,
  'word': 'Walt',
  'start': 54,
  'end': 58},
 {'entity': 'I-location',
  'score': 0.99011505,
  'index': 14,
  'word': '##eng',
  'start': 58,
  'end': 61},
 {'entity': 'I-location',
  'score': 0.9840162,
  'index': 15,
  'word': '##oo',
  'start': 61,
  'end': 63},
 {'entity': 'I-location',
  'score': 0.99047637,
  'index': 16,
  'word': 'Na',
  'start': 64,
  'end': 66},
 {'entity': 'I-location',
  'score': 0.9795417,
  'index': 17,
  'word': '##r',
  'start': 66,
  'end': 67}]

We can manually get the classification results step by step.

Tokenize the text and return PyTorch tensors:

In [41]:
inputs = tokenizer(" ".join(text), return_tensors="pt", padding=True)
for key in inputs:
    print(key + ": ", inputs[key])

input_ids:  tensor([[  101,   111,   176,  1204,   132,   115,  3284,  1314,  1989, 13776,
          2908, 12453,  1121, 10495, 14429,  5658, 11896,  1197,  1187, 10366,
          1127,  1841,  1170,   170,  1326,  1104,   170,  7501, 23742,  1116,
          1855,  1103,  1298,  1107,  1478,  1107,  1103,  1588,  1104,  1103,
          3441,   119,   102]])
token_type_ids:  tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:  tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


Pass the inputs to the model and return the `logits`:

In [33]:
import torch

In [37]:
with torch.no_grad():
    logits = model(**inputs).logits

Get the class with the highest probability, and use the model's `id2label` mapping to convert it to a text label:

In [38]:
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'I-location',
 'I-location',
 'I-location',
 'I-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

Great! We have fine tuned the bert-base-NER model on the WNUT 2017 dataset. Next, we will experiment with evaluation metrics, training parameters, and saving model in the Hugging Face hub.

## Define Customized Evaluation Metrics

To better evaluate the model's performance during training, we should pass our own evaluation metrics.

You will leverage the Huggingface [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, we will load the [seqeval](https://huggingface.co/spaces/evaluate-metric/seqeval) framework (see the Huggingface Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric).

Seqeval produces several scores: precision, recall, F1, and accuracy.

Install evaluate and seqeval.

In [42]:
! pip install -q evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


Import evaluate and create a seqeval instance.

In [43]:
import evaluate

seqeval = evaluate.load("seqeval")

Define a function `compute_metrics` that can be used in Trainer. The function will receive the results generated by the model in a batch mode. We will unpack the results and compute the metrics.

In [44]:
def compute_metrics(p):
    # p is the results containing a list of predictions and a list of labels
    # Unpack the predictions and true labels from the input tuple 'p'.
    predictions_list, labels_list = p

    # Convert the raw prediction scores into tag indices by selecting the tag with the highest score for each token.
    predictions_list = np.argmax(predictions_list, axis=2)

    # Filter out the '-100' labels that were used to ignore certain tokens (like sub-tokens or special tokens).
    # Convert the numeric tags in 'predictions' and 'labels' back to their string representation using 'tag_names'.
    # Only consider tokens that have tags different from '-100'.
    true_predictions = [
        [tag_names[p] for (p, l) in zip(predictions, labels) if l != -100]
        for predictions, labels in zip(predictions_list, labels_list)
    ]
    true_tags = [
        [tag_names[l] for (p, l) in zip(predictions, labels) if l != -100]
        for predictions, labels in zip(predictions_list, labels_list)
    ]

    # Evaluate the predictions using the 'seqeval' library, which is commonly used for sequence labeling tasks like NER.
    # This provides metrics like precision, recall, and F1 score for sequence labeling tasks.
    results = seqeval.compute(predictions=true_predictions, references=true_tags)

    # Return the evaluated metrics as a dictionary.
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


## Define TrainingArguments

We'll push this model to the Hub by setting `push_to_hub=True`. Let us log into our Hugging Face hub.

In [46]:
from huggingface_hub import notebook_login
notebook_login()

Define a TrainingArguments instance with different parameters.

In [47]:
training_args = TrainingArguments(
    output_dir="my_finetuned_wnut_model_1012",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

## Train the Model with the TrainingArguments Instance and the Customized Evaluation Metrics

First, reload the original model and tokenizer from bert-base-NER.

In [57]:
model = AutoModelForTokenClassification.from_pretrained(
    "dslim/bert-base-NER", num_labels=len(id2label), id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([13]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768])

Let us fine tune it with the metrics and training parameters.

In [50]:
import numpy as np

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.265722,0.515663,0.396664,0.448402,0.946824
2,No log,0.294021,0.547927,0.392030,0.457050,0.948705


TrainOutput(global_step=426, training_loss=0.05979704073337322, metrics={'train_runtime': 115.4266, 'train_samples_per_second': 58.808, 'train_steps_per_second': 3.691, 'total_flos': 213363894731352.0, 'train_loss': 0.05979704073337322, 'epoch': 2.0})

Once training is completed, we can share the model in the hub with the [push_to_hub()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.push_to_hub) method so everyone can use the model:

In [51]:
trainer.push_to_hub()

'https://huggingface.co/anyuanay/my_finetuned_wnut_model_1012/tree/main/'

## Inference
We can do an inference as before by loading the fine-tuned model from the hub.

In [52]:
text = wnut['test'][1]['tokens']
text

['&',
 'gt',
 ';',
 '*',
 'Police',
 'last',
 'week',
 'evacuated',
 '80',
 'villagers',
 'from',
 'Waltengoo',
 'Nar',
 'where',
 'dozens',
 'were',
 'killed',
 'after',
 'a',
 'series',
 'of',
 'avalanches',
 'hit',
 'the',
 'area',
 'in',
 '2005',
 'in',
 'the',
 'south',
 'of',
 'the',
 'territory',
 '.']

Load the fine tuned model and tokenizer from the hub.

In [53]:
model = AutoModelForTokenClassification.from_pretrained("anyuanay/my_finetuned_wnut_model_1012")
tokenizer = AutoTokenizer.from_pretrained("anyuanay/my_finetuned_wnut_model_1012")

classifier = pipeline("ner", model=model, tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Classify the text.

In [55]:
classifier(" ".join(text))

[{'entity': 'B-location',
  'score': 0.95226973,
  'index': 13,
  'word': 'Walt',
  'start': 54,
  'end': 58},
 {'entity': 'I-location',
  'score': 0.92469627,
  'index': 14,
  'word': '##eng',
  'start': 58,
  'end': 61},
 {'entity': 'I-location',
  'score': 0.91868263,
  'index': 15,
  'word': '##oo',
  'start': 61,
  'end': 63},
 {'entity': 'I-location',
  'score': 0.934662,
  'index': 16,
  'word': 'Na',
  'start': 64,
  'end': 66},
 {'entity': 'I-location',
  'score': 0.63156825,
  'index': 17,
  'word': '##r',
  'start': 66,
  'end': 67}]

Great! We have fine-tuned the pre-trained bert-base-NER model with customized metrics and different training parameters.